Import libraries & define paths

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives

In [ ]:
## More directories paths defined
train_dir = os.path.join(path, 'train')
validation_dir = os.path.join(path, 'validation')
test_dir = os.path.join(path, 'test')

## Directory with training of WBCs datasets
train_healthy_dir = os.path.join(train_dir, 'healthy')
train_ALL_dir = os.path.join(train_dir, 'ALL')

## Directory with validation of WBCs datasets
validation_healthy_dir = os.path.join(validation_dir, 'healthy')
validation_ALL_dir = os.path.join(validation_dir, 'ALL')

## Directory with testing of WBCs datasets
test_healthy_dir = os.path.join(test_dir, 'healthy')  
test_ALL_dir = os.path.join(test_dir, 'ALL') 

## Batch size
TRAIN_BATCH_SIZE = 16
VALIDATION_BATCH_SIZE = 13
TEST_BATCH_SIZE = 13

## Input image width, height and shape
IMG_HEIGHT = 150
IMG_WIDTH = 150
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

## Total number of different datasets
num_training_healthy = len(os.listdir(train_healthy_dir))
num_training_ALL = len(os.listdir(train_ALL_dir))

num_validation_healthy = len(os.listdir(validation_healthy_dir))
num_validation_ALL = len(os.listdir(validation_ALL_dir))

num_test_healthy = len(os.listdir(test_healthy_dir))
num_test_ALL = len(os.listdir(test_ALL_dir))

total_train = num_training_ALL + num_training_healthy
total_validation = num_validation_ALL + num_validation_healthy
total_test = num_test_ALL + num_test_healthy

print("Size of training dataset: " , total_train)
print("Size of validation dataset: " , total_validation)
print("Size of test dataset: " , total_test)

Download pretrained InceptionV3 model

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

View architecture of InceptionV3 model

In [ ]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = IMG_SHAPE, 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary()

Display architecture of pretrained InceptionV3 model

In [ ]:
layers = [(layer, layer.name, layer.trainable) for layer in pre_trained_model.layers]
pre_trained_model_architecture = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', pre_trained_model_architecture.shape[0]+1)
pre_trained_model_architecture

Feature Extraction

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

def extract_features(directory, sample_count, batch_size):
    features = np.zeros(shape=(sample_count, 3, 3, 2048))
    labels = np.zeros(shape=(sample_count))

    generator = datagen.flow_from_directory(directory,
                                            target_size=(IMG_WIDTH, IMG_HEIGHT),
                                            batch_size = batch_size,
                                            class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = pre_trained_model.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, total_train, TRAIN_BATCH_SIZE)  
validation_features, validation_labels = extract_features(validation_dir, total_validation, VALIDATION_BATCH_SIZE)
test_features, test_labels = extract_features(test_dir, total_test, TEST_BATCH_SIZE)

Define new classifier

In [ ]:
new_classifier = Sequential([
  GlobalAveragePooling2D(input_shape=(3, 3, 2048)),
  Dense(1, activation='sigmoid')
])

new_classifier.compile(
    optimizer = Adam(lr=0.0001/10), 
    loss = BinaryCrossentropy(),
    metrics = [BinaryAccuracy(), Precision(), Recall(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()]
)

new_classifier.summary()

Train newly defined classifier

In [ ]:
checkpoint = ModelCheckpoint(
        filepath=file_p
        monitor='val_loss',
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
        save_freq='epoch'
)

new_classifier_history = new_classifier.fit(
                                train_features, train_labels,
                                epochs=300,
                                batch_size=TRAIN_BATCH_SIZE,
                                steps_per_epoch = len(train_features) // TRAIN_BATCH_SIZE,
                                verbose=2, 
                                callbacks = [checkpoint],
                                validation_data=(validation_features, validation_labels),
                                validation_steps = len(validation_features) // VALIDATION_BATCH_SIZE
                        )

Visualize results 

In [ ]:
## Visualize training results
acc = new_classifier_history.history['binary_accuracy']
val_acc = new_classifier_history.history['val_binary_accuracy']

loss = new_classifier_history.history['loss']
val_loss = new_classifier_history.history['val_loss']

epochs_range = range(1,301)

## Plot the graph of training and validation accuracy
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.ylim(top=1.0) 
plt.ylim(bottom=0.0)
plt.axvline(x=95, color='black')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

## Plot the graph of training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.ylim(top=2.0) 
plt.ylim(bottom=0.0)
plt.axvline(x=95, color='black')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Best checkpoint
'''

Epoch 95/300
13/13 - 0s - loss: 0.4906 - binary_accuracy: 0.8173 - precision_53: 0.8000 - recall_53: 0.8462 - true_positives_53: 88.0000 - true_negatives_53: 82.0000 - false_positives_53: 22.0000 - false_negatives_53: 16.0000 - 
val_loss: 0.5274 - val_binary_accuracy: 0.8077 - val_precision_53: 0.7857 - val_recall_53: 0.8462 - val_true_positives_53: 11.0000 - val_true_negatives_53: 10.0000 - val_false_positives_53: 3.0000 - val_false_negatives_53: 2.0000

'''

Evaluate new classifier

In [ ]:
## Define architecture of model
final_classifier = Sequential([
  GlobalAveragePooling2D(input_shape=(3, 3, 2048)),
  Dense(1, activation='sigmoid')
])

## Load selected trained weights of classifier
final_classifier.load_weights(load_weights_path)

## Freeze all layers
final_classifier.trainable = False

## Compile the new classifier
final_classifier.compile(
    optimizer = Adam(lr=0.0001/10), 
    loss = BinaryCrossentropy(),
    metrics = [BinaryAccuracy(), Precision(), Recall(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()]
)

final_classifier.summary()

# Carry out testing of the classifier
test_history = final_classifier.evaluate(
    test_features, test_labels,
    batch_size=TEST_BATCH_SIZE,
    verbose=1,
    steps=len(test_features) // TEST_BATCH_SIZE
)

# Evaluation results:
# loss: 0.4982 - binary_accuracy: 0.8974 - precision_1: 0.9444 - recall_1: 0.8308 - true_positives_1: 8.6667 - 
# true_negatives_1: 10.6667 - false_positives_1: 0.6667 - false_negatives_1: 1.6667

Model Finetuning

In [ ]:
train_val_image_gen = ImageDataGenerator(
                    rotation_range=180,
                    horizontal_flip=True,
                    vertical_flip=True,
                    preprocessing_function=preprocess_input
                  )

test_image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data_gen = train_val_image_gen.flow_from_directory(batch_size=TRAIN_BATCH_SIZE, 
                                                     directory=train_dir, 
                                                     shuffle=True, 
                                                     target_size=(IMG_WIDTH, IMG_HEIGHT), 
                                                     class_mode='binary')

validation_data_gen = train_val_image_gen.flow_from_directory(batch_size=VALIDATION_BATCH_SIZE, 
                                                          directory=validation_dir, 
                                                          shuffle=True, 
                                                          target_size=(IMG_WIDTH, IMG_HEIGHT), 
                                                          class_mode='binary')

test_data_gen = test_image_gen.flow_from_directory(batch_size=TEST_BATCH_SIZE,
                                                              directory=test_dir,
                                                              target_size=(IMG_WIDTH, IMG_HEIGHT), 
                                                              class_mode='binary')

Build final classifier

In [ ]:
final_classifier = Sequential([
  GlobalAveragePooling2D(input_shape=(3, 3, 2048)),
  Dense(1, activation='sigmoid')
])

final_classifier.load_weights(final_load_weights_path)

In [ ]:
## Pre-trained InceptionV3 as based model
based_model = InceptionV3(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

## Unfreeze all layers of InceptionV3
based_model.trainable = True

## Refreeze layers until layers plan to fine-tune 
for layer in based_model.layers[:268]:
  layer.trainable = False

## Display the architecture of InceptionV3
layers = [(layer, layer.name, layer.trainable) for layer in based_model.layers]
based_model_architecture = pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', based_model_architecture.shape[0]+1)
based_model_architecture

Define final classifier

In [ ]:
model = Sequential([
  based_model,
  final_classifier
])

model.compile(
    optimizer = Adam(lr=0.0001/10), 
    loss = BinaryCrossentropy(),
    metrics = [BinaryAccuracy(), Precision(), Recall(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()]
)


model.summary()

Train final classifier model

In [ ]:
checkpoint = ModelCheckpoint(
        filepath=file_p,
        monitor='val_loss',
        save_best_only=False,
        save_weights_only=False,
        verbose=1,
        save_freq='epoch'
)

model_history = model.fit(
                    train_data_gen,
                    epochs = 100,
                    steps_per_epoch = train_data_gen.samples // TRAIN_BATCH_SIZE,
                    verbose = 2, 
                    callbacks = [checkpoint],
                    validation_data = validation_data_gen,
                    validation_steps = validation_data_gen.samples // VALIDATION_BATCH_SIZE
                )

Visualize results

In [ ]:
acc = model_history.history['binary_accuracy']
val_acc = model_history.history['val_binary_accuracy']

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs_range = range(1,101)

## Plot the graph of training and validation accuracy
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.ylim(top=1.0) 
plt.ylim(bottom=0.0)
plt.axvline(x=20, color='black')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

## Plot the graph of training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.ylim(top=2.0) 
plt.ylim(bottom=0.0)
plt.axvline(x=20, color='black')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Best Checkpoint
'''

Epoch 20/100
13/13 - 14s - loss: 0.2028 - binary_accuracy: 0.9327 - precision_2: 0.9412 - recall_2: 0.9231 - true_positives_2: 96.0000 - true_negatives_2: 98.0000 - false_positives_2: 6.0000 - false_negatives_2: 8.0000 - 
val_loss: 0.2941 - val_binary_accuracy: 0.9231 - val_precision_2: 0.9231 - val_recall_2: 0.9231 - val_true_positives_2: 12.0000 - val_true_negatives_2: 12.0000 - val_false_positives_2: 1.0000 - val_false_negatives_2: 1.0000

'''

Evaluate Finetuned classifier

In [ ]:
from tensorflow.keras.models import load_model

final_model = load_model(final_weights_path)
final_model.trainable = False

final_model.compile(
    optimizer = Adam(lr=0.0001/10),
    loss = BinaryCrossentropy(),
    metrics = [BinaryAccuracy(), Precision(), Recall(), TruePositives(), TrueNegatives(), FalsePositives(), FalseNegatives()]
)

test_history = final_model.evaluate(
    test_data_gen,
    verbose=1,
    steps=test_data_gen.samples // TEST_BATCH_SIZE
)

Testing final classifier

In [ ]:
test_history = final_model.evaluate(
    test_data_gen,
    verbose=1,
    steps=test_data_gen.samples // TEST_BATCH_SIZE
)

#Evaluation Results:
#loss: 0.2229 - binary_accuracy: 0.9615 - precision_4: 1.0000 - recall_4: 0.9231 - 
#true_positives_4: 12.0000 - true_negatives_4: 13.0000 - false_positives_4: 0.0000e+00 - false_negatives_4: 1.0000

Sample classification

In [ ]:
pred_dir = os.path.join(path, 'prediction')

predict_data_gen = test_image_gen.flow_from_directory(batch_size=1,
                                                        directory=pred_dir,
                                                        target_size=(IMG_WIDTH, IMG_HEIGHT), 
                                                        class_mode='binary')

fnames = predict_data_gen.filenames
ground_truth = predict_data_gen.classes
label2index = predict_data_gen.class_indices
idx2label = dict((v,k) for k,v in label2index.items())

print(fnames)
print(ground_truth)
print(label2index)
print(idx2label)

In [ ]:
from tensorflow.keras.preprocessing import image

predictions = final_model.predict(predict_data_gen)

for i in range(len(predictions)):
    pred_label = idx2label[int(predictions[i] + 0.5)]
    actual_label = idx2label[ground_truth[i]]
    
    title = '{}, Prediction: {}, Actual label: {}, Predicted label: {}'.format(fnames[i], predictions[i], actual_label, pred_label)
    
    actual_img = image.load_img('{}/{}'.format(test_dir,fnames[i]))
    plt.figure(figsize=[5,5])
    plt.axis('off')
    plt.title(title)
    plt.imshow(actual_img)
    plt.show()